**Process pedestrian JSON file**

In [1]:
%matplotlib inline  
import os
import numpy as np
from PIL import Image
import cv2
from matplotlib.pyplot import figure, imshow, savefig
import matplotlib.pyplot as plt
import collections
import os
import json

In [4]:
fold_dict_filename = 'pedestrian_dataset_folds/fold_dict.json'
fold_dict = json.load(open(fold_dict_filename, 'r'))
num_frames = 30
frames_to_process = set()
for json_filename in fold_dict:
    json_path = os.path.join(fold_dict[json_filename], json_filename)
    ped_json = json.load(open(json_path, 'r'))
    video_name = ped_json['video']
    first_frame = ped_json['frame_data'][0]
    start = first_frame['frame_index']
    for idx in range(start, start + num_frames):
        frames_to_process.add(video_name + '-' + str(idx))